# Old integration code

In [ ]:
from __future__ import print_function
from IPython.display import display, clear_output
import sys, os
import time
import math
import getdist.plots as gplot
from sympy import *
from sympy.solvers import solve
from scipy import optimize
from scipy.constants import physical_constants
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from matplotlib import pyplot as plt
import scipy.integrate as integrate
import numpy as np

init_vprinting()

phi_ = Symbol('phi')
Phi_ = Function('\Phi')(phi_)
xi_ = Symbol('xi')
m_ = Symbol('m')
Mp_ = Symbol('M_{p}')

U = 0.5*m_**2*phi_**2
Om = 1 + xi_*phi_**2/Mp_**2
dPdp2 = 1/Om + 3/2 * Mp_**2 * (diff(Om,phi_)/Om)**2
dPdp = sqrt(1/Om + 3/2 * Mp_**2 * (diff(Om,phi_)/Om)**2)
V = U/Om**2
V_prime = diff(V,phi_)*(dPdp**(-1))
V_pprime = diff(V_prime, phi_) * (dPdp**(-1))

eps = Mp_**2/2 *(V_prime/V)**2
eta = Mp_**2*(V_pprime/V)

phi_f = solve(eps-1, phi_)
phi_f[3]

xi_ = np.linspace(-0.002,0.006,1000)
Mp = physical_constants['Planck mass'][0]
Mp = 4.341e-9


def epsi(phi, xi):
    return 2.0*(Mp**2 - phi**2*xi)**2/(phi**2*(Mp**2 + 6.0*phi**2*xi**2 + phi**2*xi))

def etaa(phi, xi):
    return (2.0*Mp**6 - 12.0*Mp**4*phi**2*xi - 72.0*Mp**2*phi**4*xi**3 - 10.0*Mp**2*phi**4*xi**2 + 24.0*phi**6*xi**4 + 
            4.0*phi**6*xi**3)/(phi**2*(1.0*Mp**4 + 12.0*Mp**2*phi**2*xi**2 + 2.0*Mp**2*phi**2*xi + 36.0*phi**4*xi**4 + 
                                       12.0*phi**4*xi**3 + 1.0*phi**4*xi**2))

def integrand_(phi, xi):
    return 0.5*phi*(Mp**2 + 6*phi**2*xi**2 + phi**2*xi)/(Mp**4 - phi**4*xi**2)

def phi_f(xi):
    return np.sqrt(Mp**2*(np.sqrt(48*(xi**2) + 16*xi + 1) -1 -4*xi)/(8*(xi**2) + 2*xi))

def trapezoid(f, xi, a, b, n):
    dx = (b-a)/n
    res = 0
    for i in range(1,n):
        res += f(a + i*dx, xi)
    res += 0.5*(f(a, xi) + f(b, xi))
    return res*dx

Dx = 1e-11
tol = 1e-3
counter = 0

phi_i_50 = []
phi_i_60 = []
print("Starting calculation of initial scalar field with 60 e-folding...\n")
for xi in xi_:
    N = 0
    i = 1
    counter += 1
    while abs(N-60) >= tol:
        N = trapezoid(integrand_, xi, phi_f(xi), phi_f(xi)+i*Dx, 1000)
        i += 1
    print(counter)
    phi_i_60.append(phi_f(xi)+(i-1)*Dx)
print("Finished calculating for 60 e-folding...\n")
counter = 0
print("Starting calculation of initial scalar field with 50 e-folding...\n")
for xi in xi_:
    N = 0
    i = 1
    counter += 1
    while abs(N-50) >= tol:
        N = trapezoid(integrand_, xi, phi_f(xi), phi_f(xi)+i*Dx, 1000)
        i += 1
    print(counter)
    phi_i_50.append(phi_f(xi)+(i-1)*Dx)
print("Calculation of the initial scalar field finished...\n")


_eta_60 = []
_eps_60 = []
_eta_50 = []
_eps_50 = []

for i in range(len(phi_i_60)):
    _eta_60.append(etaa(phi_i_60[i], xi_[i]))
    _eps_60.append(epsi(phi_i_60[i], xi_[i]))
    _eta_50.append(etaa(phi_i_50[i], xi_[i]))
    _eps_50.append(epsi(phi_i_50[i], xi_[i]))

ns_60 = []
r_60 = []
ns_50 = []
r_50 = []
for i in range(len(phi_i_60)):
    ns_60.append(1 + 2*_eta_60[i] - 6*_eps_60[i])
    r_60.append(16*_eps_60[i])
    ns_50.append(1 + 2*_eta_50[i] - 6*_eps_50[i])
    r_50.append(16*_eps_50[i])    
    
    
%matplotlib qt

g = gplot.getSinglePlotter(chain_dir=[r'/Volumes/Newton/planck/2018/COM_CosmoParams_fullGrid_R3.00/base_r/plikHM_TT_lowl_lowE',
                                      r'/Volumes/Newton/planck/2018/COM_CosmoParams_fullGrid_R3.00/base_r/plikHM_TTTEEE_lowl_lowE',
                                      r'/Volumes/Newton/planck/2018/COM_CosmoParams_fullGrid_R3.00/base_r/CamSpecHM_TTTEEE_lowl_lowE_lensing',
                                      r'/Volumes/Newton/planck/2015/COM_CosmoParams_fullGrid_R2.00/base_r/plikHM_TT_lowTEB'])
g.settings.legend_frame = False
g.settings.legend_loc = 'best'
g.settings.figure_legend_frame = False

roots = []
roots.append('base_r_plikHM_TT_lowl_lowE')
roots.append('base_r_plikHM_TT_lowl_lowE_post_BAO')
roots.append('base_r_plikHM_TTTEEE_lowl_lowE')
roots.append('base_r_CamSpecHM_TTTEEE_lowl_lowE_lensing')
roots.append('base_r_plikHM_TT_lowTEB')
roots.append('base_r_plikHM_TT_lowTEB_post_BAO')
pairs = [('ns','r')]

g.plots_2d(roots, param_pairs=pairs, legend_labels=[], filled=True, shaded=False)
g.add_line([0.96694214876], [0.132231404959], label=['N = 60'], ls='None', zorder=3, color='red', marker='o', markeredgewidth=7)
g.add_line([0.960396039604], [0.158415841584], label=['N = 50'], ls='None', zorder=3, color='red', marker='o', markeredgewidth=2)

leg1 = g.add_legend(['Planck TT','Planck TT + BAO','Planck TTTEEE','CamSpec TTTEEE + Lensing','Planck 2015 TT', 'Planck 2015 TT + BAO'], colored_text=True, fontsize=13, legend_loc='upper left', figure=False)
leg2 = g.subplots[0,0].legend(['N = 60', 'N = 50'], loc='upper right', frameon=False)
g.subplots[0,0].add_artist(leg1)

plt.scatter(x60, y60, c=xi_, s=1, cmap = 'inferno', vmin=xi_.min(), vmax=xi_.max())
plt.scatter(x50, y50, c=xi_, s=1, cmap = 'inferno', vmin=xi_.min(), vmax=xi_.max())

abs_xi = [abs(xi_[i]) for i in range(len(xi_))]
print("Error in calculations comparing with minimal coupling \n ")
print(50*"-" + "\n")
print("60 e-fold error = " + str(abs(x60[np.argmin(abs_xi)] - 0.96694214876)/0.96694214876 * 100) + "%")
print(50*"-" + "\n")
print("50 e-fold error = " + str(abs(x50[np.argmin(abs_xi)] - 0.960396039604)/0.960396039604 * 100) + "%")

plt.colorbar(label = r'$\xi$')
plt.xlim(0.94,0.98)
plt.ylim(0.00,0.20)

g.export('nm.pdf',adir='coupling/150205193/figures/')